In [3]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint

In [4]:
def prepare_images(path, factor):
    
    # loop through the file in the directory 
    for file in os.listdir(path):
        img = cv2.imread(path + '/' + file)
        
        # find old and new image dimensions
        h, w, c = img.shape
        new_height = int(h / factor)
        new_width = int(w / factor)
        
        # resize the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_LINEAR)
        
        # resize the image - up
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)
        
        # save the image
        print('Saving {}'.format(file))
        cv2.imwrite('./images/{}'.format(file), img)

In [5]:
prepare_images('./source', 2)

Saving baboon.bmp
Saving barbara.bmp
Saving bridge.bmp
Saving coastguard.bmp
Saving comic.bmp
Saving face.bmp
Saving flowers.bmp
Saving foreman.bmp
Saving lenna.bmp
Saving man.bmp
Saving monarch.bmp
Saving pepper.bmp
Saving ppt3.bmp
Saving zebra.bmp


In [76]:
def image_data(path):
    names = os.listdir(path)   # 100개의 데이터를 순서대로 불러오기
    names = sorted(names)       # 100개의 데이터가 순서대로 정리
    nums = names.__len__()      # 100
    data = []
    for i in range(nums):
        name = path + names[i]
        img = cv2.imread(name)
        data.append(img)
    return data
    # for file in os.listdir(path):
    #     target = []
    #     img = cv2.imread(path + '/' + file)
    #     target.append(img)
    # return np.array(target)

In [91]:
x = image_data('./source/')
y = image_data('./images/')


In [95]:
print(x[0].shape, y[0].shape)

(480, 500, 3) (480, 500, 3)


In [13]:
def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 0:sz[1]]
    return img

def shave(image, border):
    img = image[border:-border, border:-border]
    return img

In [25]:

BLOCK_STEP = 16
BLOCK_SIZE = 32
Random_Crop = 30
Patch_size = 32
label_size = 20
conv_side = 6

def prepare_crop_data(_path):
    names = os.listdir(_path)
    names = sorted(names)
    nums = names.__len__()

    data = []
    label = []

    for i in range(nums):
        name = _path + names[i]
        hr_img = cv2.imread(name, cv2.IMREAD_COLOR)
        hr_img = cv2.cvtColor(hr_img, cv2.COLOR_BGR2YCrCb)
        hr_img = hr_img[:, :, 0]
        shape = hr_img.shape
        
        scale = 2
        w, h, = hr_img.shape
        new_width = int(w / scale)
        new_height = int(h / scale)
        
        lr_img = cv2.resize(hr_img, (new_width, new_height))
        lr_img = cv2.resize(lr_img, (shape[1], shape[0]))

        width_num = (shape[0] - (BLOCK_SIZE - BLOCK_STEP) * 2) / BLOCK_STEP
        height_num = (shape[1] - (BLOCK_SIZE - BLOCK_STEP) * 2) / BLOCK_STEP
        for k in range(width_num):
            for j in range(height_num):
                x = k * BLOCK_STEP
                y = j * BLOCK_STEP
                hr_patch = hr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]
                lr_patch = lr_img[x: x + BLOCK_SIZE, y: y + BLOCK_SIZE]

                lr_patch = lr_patch.astype(float) / 255.
                hr_patch = hr_patch.astype(float) / 255.

                lr = np.zeros((1, Patch_size, Patch_size), dtype=np.double)
                hr = np.zeros((1, label_size, label_size), dtype=np.double)

                lr[0, :, :] = lr_patch
                hr[0, :, :] = hr_patch[conv_side: -conv_side, conv_side: -conv_side]

                data.append(lr)
                label.append(hr)

    data = np.array(data, dtype=float)
    label = np.array(label, dtype=float)
    return data, label

In [26]:
datasets = prepare_crop_data('HR/')

datasets[0].shape
# label = datasets[1]

TypeError: 'float' object cannot be interpreted as an integer

In [22]:
data[0].shape

(384, 384, 3)

In [ ]:
SRCNN = Sequential()
    
# add model layers
SRCNN.add(Conv2D(filters=128, kernel_size=(9, 9), kernel_initializer='glorot_uniform',
                     activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
SRCNN.add(Conv2D(filters=64, kernel_size=(3, 3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
SRCNN.add(Conv2D(filters=1, kernel_size=(5,5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='valid', use_bias=True))
    
# compile model
SRCNN.compile(optimizer='adam', loss='mse', metrics=['mse'])
SRCNN.fit(data, label, batch_size=10, validation_data=(val_data, val_label), epochs=200, verbose=1)

In [37]:
original_img = cv2.imread('./source/baboon.bmp')
img.shape

(480, 500, 3)